In [1]:
import pandas as pd
import nltk
import string
# Download the NLTK tokenizer and positive word set
nltk.download('punkt')
nltk.download('cmudict')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ANIKET\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\ANIKET\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [2]:
df_fetched = pd.read_csv("data/sources_exported.csv")
df_fetched.head()

,URL_ID,URL,contents
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,"Telemedicine, the use of technology to diagnos..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,"The rise of e-health, or the use of electronic..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,"“More gains on quality, affordability and acce..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,"“More gains on quality, affordability and acce..."


In [3]:
df_original = pd.read_csv("data/sources.csv")
df_original.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df_original.merge(df_fetched, how = "outer")
df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,contents
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Telemedicine, the use of technology to diagnos..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The rise of e-health, or the use of electronic..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"“More gains on quality, affordability and acce..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"“More gains on quality, affordability and acce..."


In [5]:
# Read positive words
PATH_POSITIVE_WORDS = "data/positive-words.txt"
PATH_NEGATIVE_WORDS = "data/negative-words.txt"

def read_words(path_to_file):
    with open(path_to_file, "r") as file:
        lines = file.readlines()
        word_set = {word.strip() for word in lines[35::]}
        
        return word_set
    
positive_words = read_words(PATH_POSITIVE_WORDS)
negative_words = read_words(PATH_NEGATIVE_WORDS)

In [6]:
def calculate_positive_score(tokens):
    return sum(1 for word in tokens if word in positive_words)

def calculate_negative_score(tokens):
    return sum(1 for word in tokens if word in negative_words)

def calculate_subjectivity_score(positive_score, negative_score, total_words):
    # Add a small value (0.000001) to avoid division by zero
    denominator = (total_words) + 0.000001
    subjectivity_score = (positive_score + negative_score) / denominator
    return subjectivity_score

def calculate_polarity_score(positive_score, negative_score):
    # Add a small value (0.000001) to avoid division by zero
    denominator = (positive_score + negative_score) + 0.000001
    polarity_score = (positive_score - negative_score) / denominator
    return polarity_score


def calculate_average_sentence_length(tokens):
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(tokens)
    
    # Calculate the total number of words
    total_words = sum(len(nltk.word_tokenize(sentence)) for sentence in sentences)
    
    # Calculate the total number of sentences
    total_sentences = len(sentences)
    
    # Calculate average sentence length
    average_sentence_length = total_words / total_sentences if total_sentences > 0 else 0
    return average_sentence_length
def count_complex_words(text):
    # Exclude punctuation from the text
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    
    # Tokenize the text into words
    words = nltk.word_tokenize(text_without_punctuation.lower())
    
    # Initialize a counter for complex words
    complex_words_count = 0
    
    # Load the CMU Pronouncing Dictionary for syllable count
    cmu_dict = nltk.corpus.cmudict.dict()
    
    for word in words:
        # Check if the word has more than two syllables
        if word in cmu_dict and max([len(list(y)) for y in cmu_dict[word]]) > 2:
            complex_words_count += 1
    
    return complex_words_count

def calculate_percentage_complex_words(total_words, complex_words_count):
    percentage_complex_words = (complex_words_count / total_words) * 100 if total_words > 0 else 0
    return percentage_complex_words

def calculate_fog_index(average_sentence_length, percentage_complex_words):
    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)
    return fog_index

# Example text for analysis
example_text = "This is a great day! I'm feeling wonderful."

# Tokenize the text into words
tokens_eg = nltk.word_tokenize(example_text.lower())

# Calculate the positive and negative scores
positive_score_eg = calculate_positive_score(tokens_eg)
negative_score_eg = calculate_negative_score(tokens_eg)

# Calculate the subjectivity score
total_words_eg = len(tokens_eg)
subjectivity_score_rg = calculate_subjectivity_score(positive_score_eg, negative_score_eg, total_words_eg)

# Calculate the polarity score
polarity_score_eg = calculate_polarity_score(positive_score_eg, negative_score_eg)

average_sentence_length_rg = calculate_average_sentence_length(tokens_eg)

complex_words_count_rg = count_complex_words(example_text)

# Calculate the percentage of complex words
total_words = len(nltk.word_tokenize(example_text.lower()))
percentage_complex_words_eg = calculate_percentage_complex_words(total_words, complex_words_count)

# Calculate the fog index
fog_index_eg = calculate_fog_index(average_sentence_length_rg, percentage_complex_words)

# Display the scores
print("Positive Score:", positive_score_eg)
print("Negative Score:", negative_score_eg)
print("Polarity Score:", polarity_score_eg)
print("Subjectivity Score:", subjectivity_score_rg)
print("complex_words_count:", total_words_eg)
print("average_sentence_length:",average_sentence_length_rg)
print("complex_words_count:", complex_words_count_rg )
print("percentage_complex_words:", percentage_complex_words_eg)
print("fog_index:", fog_index_eg)


TypeError: expected string or bytes-like object

In [ ]:
def find_params(text):
    
    try:
        text = text.lower()
    except Exception as e:
        text = ""
        
    tokens = nltk.word_tokenize(text.lower())
        
    # Calculate the positive and negative scores
    positive_score = calculate_positive_score(tokens)
    negative_score = calculate_negative_score(tokens)

    # Calculate the subjectivity score
    total_words = len(tokens)
    subjectivity_score = calculate_subjectivity_score(positive_score, negative_score, total_words)

    # Calculate the polarity score
    polarity_score = calculate_polarity_score(positive_score, negative_score)
    
    #Calculate average_sentence
    
    avg_sentence_length  = calculate_average_sentence_length(total_words)
    
    perc_complex = None
    
    fog_index = None
    
    avg_num_words = None
    
    complex_word_count = None
    
    word_count = None
    
    syllable_per_word = None
    
    personal_pronouns = None
    
    avg_word_length = None
    
    return (
        positive_score, 
        negative_score, 
        polarity_score, 
        subjectivity_score, 
        avg_sentence_length, 
        perc_complex, 
        fog_index, 
        avg_num_words, 
        complex_word_count, 
        word_count, 
        syllable_per_word, 
        personal_pronouns, 
        avg_word_length
        )

In [ ]:
params = df.apply(lambda row: find_params(row["contents"]), axis=1)
df.iloc[:, 2:15] = pd.DataFrame(params.to_list())

TypeError: calculate_average_sentence_length() missing 1 required positional argument: 'negative_score'

In [ ]:
df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,contents
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,88.0,24.0,0.571429,0.060969,None,None,None,None,None,None,None,None,None,"Telemedicine, the use of technology to diagnos..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,41.0,13.0,0.518519,0.083333,None,None,None,None,None,None,None,None,None,"The rise of e-health, or the use of electronic..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,0.0,0.0,0.000000,0.000000,None,None,None,None,None,None,None,None,None,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,43.0,27.0,0.228571,0.051699,None,None,None,None,None,None,None,None,None,"“More gains on quality, affordability and acce..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,43.0,27.0,0.228571,0.051699,None,None,None,None,None,None,None,None,None,"“More gains on quality, affordability and acce..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,8.0,7.0,0.066667,0.028463,None,None,None,None,None,None,None,None,None,Before jumping on the topic I would like to gi...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,27.0,65.0,-0.413043,0.049224,None,None,None,None,None,None,None,None,None,As the coronavirus spreads around the world an...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,96.0,32.0,0.500000,0.067905,None,None,None,None,None,None,None,None,None,"From Alibaba to Ping An and Google to Ford, co..."
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,34.0,21.0,0.236364,0.035010,None,None,None,None,None,None,None,None,None,"When\nthe British ruled India, many Indians\na..."
